---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [39]:
def date_sorter():
    ## Import Libraries ##
    
    import numpy as np
    
    
    ## Extract Dates ##
    
    # Create list of regex for different formats with non-reporting groups:
    #regex_list = [
    #    r'(?:\d{1,2}(?:/|-)\d{1,2}(?:/|-)\d{2,4})',
    #    r'(?:[A-Z]\w+\.?\s\d{1,2},?\s\d{2,4})',
    #    r'(?:\d{1,2}\s[A-Z]\w+,?\.?\s\d{2,4})',
    #    #r'(?:[A-Z]\w+\s\d{1,2}(?:st|nd|rd|th),\s\d{2,4})',#this case is not in the data
    #    r'(?:[A-Z]\w+,?\s\d{4})',
    #    r'(?:\d{1,2}/\d{2,4})',
    #    r'(?:\d{4})'
    #]
    
    # Create Series with extracted dates:
    #dates = df.str.extract('(' + '|'.join(regex_list) + ')')
    
    # Create list of regex to extract day, month and year from text:
    regex_list = [
        r'(?P<month>\d{1,2})(?:/|-)(?P<day>\d{1,2})(?:/|-)(?P<year>\d{2,4})', # 1st format
        r'(?P<month>[A-Z][a-z]+)\.?\s(?P<day>\d{1,2}),?\s(?P<year>\d{2,4})', # 2nd format
        r'(?P<day>\d{1,2})\s(?P<month>[A-Z][a-z]+),?\.?\s(?P<year>\d{2,4})', # 3rd format
        r'(?P<month>[A-Z][a-z]+),?\s(?P<year>\d{4})', # 5th format
        r'(?P<month>\d{1,2})/(?P<year>\d{2,4})', # 6th format
        r'(?P<year>\d{4})' # 7th format
    ]
    
    # Create list to store extractions:
    extractions = []
    # Loop over regex list:
    for regex in regex_list:
        extractions.append(df.str.extract(regex))
    
    # Create missing columns for some extractions:
    extractions[3]['day'] = '1'
    extractions[4]['day'] = '1'
    # Convert series to dataframe then fill:
    extractions[5] = extractions[5].to_frame()
    extractions[5]['day'] = '1'
    extractions[5]['month'] = '1'
    
    # Merge results from first to last data:
    i=1 #counter
    columns = ['day', 'month', 'year'] #column names
    dates = extractions[0].copy() # desired final dataframe
    while i < len(extractions):
        # Loop ovel all columns:
        for col in columns:
            # Fill NaN with data from next colum:
            dates[col].fillna(extractions[i][col], inplace=True)        
        # Update counter
        i += 1
    
    
    ## Treat Year Data ##
    
    # Convert 1900 dates to proper format:
    dates['year'] = dates['year'].apply(lambda x: '19'+x if len(x)==2 else x)
    
    
    ## Treat Month Data ##
    
    # To visualize months:
    #print(dates['month'].unique())
    
    # Replace wrong words with numbers:
    dates['month'] = dates['month'].replace('Since', '1')
    dates['month'] = dates['month'].replace('Age', '1')
    dates['month'] = dates['month'].replace('In', '1')
    dates['month'] = dates['month'].replace('Janaury', '1')
    dates['month'] = dates['month'].replace('Decemeber', '12')
    
    # Transformation dictionary - from text to number:
    transf = {'Jan': '1', 'January': '1',
              'Feb': '2', 'February': '2',
              'Mar': '3', 'March': '3',
              'Apr': '4', 'April': '4',
              'May': '5',
              'Jun': '6', 'June': '6',
              'Jul': '7', 'July': '7',
              'Aug': '8', 'August': '8',
              'Sep': '9', 'September': '9',
              'Oct': '10', 'October': '10',
              'Nov': '11', 'November': '11',
              'Dec': '12', 'December': '12'}
    # Apply transformation on month values:
    dates['month'] = dates['month'].apply(lambda x: transf[x] if x in transf.keys() else x)
    
    
    ## Date Object Handeling ##
    
    # Create column with proper date format:
    dates['date'] = dates['year'] + '-' + dates['month'] + '-' + dates['day']
    # Convert date colum to date object:
    # Convert index to column:
    
    # Sort based on date objects:
    
    # Return only series with sorted indices:
    return dates

# Testing:
test = date_sorter()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:37: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


In [40]:
test.head()

,month,day,year,date
0,03,25,1993,1993-03-25
1,6,18,1985,1985-6-18
2,7,8,1971,1971-7-8
3,9,27,1975,1975-9-27
4,2,6,1996,1996-2-6


In [35]:
test['year'].isnull().sum()

0

In [36]:
test['month'].unique()

array(['03', '6', '7', '9', '2', '5', '10', '3', '4', '8', '1', '07', '06',
       '11', '12', '08', '02', '09', '01', '04'], dtype=object)

In [11]:
test.iloc[150]

month    March
day         12
year      1980
Name: 150, dtype: object

In [110]:
test.str.date

AttributeError: Can only use .dt accessor with datetimelike values